In [3]:
!unzip /content/data_cc.zip -d /content

Archive:  /content/data_cc.zip
   creating: /content/data/
  inflating: /content/data/eng-fra.txt  
   creating: /content/data/names/
  inflating: /content/data/names/Arabic.txt  
  inflating: /content/data/names/Chinese.txt  
  inflating: /content/data/names/Czech.txt  
  inflating: /content/data/names/Dutch.txt  
  inflating: /content/data/names/English.txt  
  inflating: /content/data/names/French.txt  
  inflating: /content/data/names/German.txt  
  inflating: /content/data/names/Greek.txt  
  inflating: /content/data/names/Irish.txt  
  inflating: /content/data/names/Italian.txt  
  inflating: /content/data/names/Japanese.txt  
  inflating: /content/data/names/Korean.txt  
  inflating: /content/data/names/Polish.txt  
  inflating: /content/data/names/Portuguese.txt  
  inflating: /content/data/names/Russian.txt  
  inflating: /content/data/names/Scottish.txt  
  inflating: /content/data/names/Spanish.txt  
  inflating: /content/data/names/Vietnamese.txt  


# Importando las librerías necesarias

In [4]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

In [5]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [6]:
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

In [7]:
device="cuda" if torch.cuda.is_available() else "cpu"

# Cargando los data files

#### Esta clase se encargará de manejar la data (word 2 index), así como el recuento de las mismas

In [8]:
#Índices de los tokens (los codificaré en OHE)
sos_token=0
eos_token=1

In [9]:
class Idioma():
  def __init__(self,name):
    self.name=name
    self.n_words=2 #2 inicialmente por los tokens sos y eos. Así, el index de la palabra inicial será 2 --> 0 y 1 están ocupadas por SOS Y EOS

    self.word2index={} #Mapear a sus índices para crear el OHE por palabra
    self.index2word={0:"SOS",1:"EOS"}
    self.word2count={}

  def addSentence(self,sentence): #Función para iterar por las palabras dada una oración
    for word in sentence.split(" "):
      self.addWord(word)

  def addWord(self,word): #Función para crear los dicts (y mapear así con sus índices correspondientes)
    if word not in self.word2index: #Si NO está esa palabra en el dict, que la cree

      self.word2index[word]=self.n_words #Word to index
      self.index2word[self.n_words]=word #Index to word
      self.word2count[word]=1 #Dict que almacena el recuento. Como esa palabra es nueva ---> count=1

      self.n_words+=1 #Actualizo n_words (pues servirá como índice para otras palabras) :D

    else: #De lo contrario, si ya existe, solo modifico el dict que lleva el recuento de esa palabra
      self.word2count[word]+=1



#### Preprocesamiento de la data: Unicode a ASCII, remover puntuaciones y demás

In [10]:
#Unicode to ascii --> Aquí se realiza la descomposición de caracteres unicode a ascii
def Unicode2Ascii(word):
  return "".join(
      c for c in unicodedata.normalize("NFD",word)
      if unicodedata.category(c)!="Mn"
  )

In [11]:
#Normalizar las oraciones: A minúscula, eliminar espacios en blancos al inicio o final, etc
def normalizeString(s):
  s=Unicode2Ascii(s.lower().strip()) #Minúscula, eliminar esp. blancos al inicio o final

  s = re.sub(r"([.!?])", r" \1", s) #Reemplaza los . ! ? por un 'espacio antes. Es decir hola! -> hola ! (pues "(white space)\1"), donde \1 hace referencia a ese signo
  s = re.sub(r"[^a-zA-Z!?]+", r" ", s) #Los que NO sean letras y signos como ! ? -> Se remplazan por un espacio
  return s.strip()

####  Almacenar la data en 'pairs'.
Nota:
- 'reverse=False' indica que la traducción va
   English --> Other Language
- 'reverse=True'
   Other language --> English

In [12]:
def readLangs(lang1, lang2, reverse=False):
  #Lista de pairs
  pairs=[]

  #Leer el archivo y dividir (por líneas) las oraciones
  lineas=open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
    read().strip().split("\n") #Divido en línea las oraciones

  #Separando capa 'pair' --> split("\n")
  for index,linea in enumerate(lineas): #itero sobre cada pair

    #Obtengo las oraciones de cada pair
    ora1,ora2=linea.split("\t")

    #Normalizo las oraciones de cada pair
    ora1_norm,ora2_norm=normalizeString(ora1),normalizeString(ora2)

    #Agrego los pairs en una lista cada uno
    pairs.append([ora1_norm,ora2_norm])

  #pairs = [[normalizeString(s) for s in l.split('\t')] for l in lineas]

  #Si reverse=True --> inglés index 1:
  if reverse:
    pairs=[list(reversed(p)) for p in pairs]

    #Instancio objetos de la clase Idioma para cada uno de los languages
    input_idioma=Idioma(lang2)
    target_idioma=Idioma(lang1)

  #Si reverse=False --> inglés index 0:
  else:
    input_idioma=Idioma(lang1)
    target_idioma=Idioma(lang2)


  return input_idioma,target_idioma,pairs

#### Filtrar algunos tipos de oraciones:
- Oraciones con menos de 10 palabras
- Oraciones que empiecen con los prefijos indicados


In [13]:
#Max length de las oraciones
max_length=10

In [14]:
#Prefijos con los que deben iniciar las oraciones a entrenar
prefijos=(
    "i am", "i m",
    "he is", "he s",
    "she is", "she s",
    "you are", "you re",
    "we are", "we re",
    "they are", "they re"
    )

In [15]:
#Función para filtrar el tipo de oraciones especificadas
def filtrarPair(p):
  return len(p[0].split(' ')) < max_length and \
      len(p[1].split(' ')) < max_length and \
      p[1].startswith(prefijos)  #Cuidado con pair[0].startswith, porque 'reverse' puede ser True y NO ser el inglés el index=0, sino 1 OJOO

In [16]:
#Función que pasa cada par por la función filtrarPair
def filterPairs(pairs):
  return [pair for pair in pairs if filtrarPair(pair)]

#### Preparar la data completa
- Para ello, uso las funciones creadas

- Formato: [['i m', 'j ai ans'],
 ['i m ok', 'je vais bien'],
 ['i m ok', 'ca va']]

In [17]:
def PrepararData(idioma1,idioma2,reverse=False):
  #Creo los pares y objetos de las clases
  idioma_input,idioma_target,pares=readLangs(idioma1,idioma2,reverse)
  print("Read %s sentence pairs" % len(pares))
  #Pares filtrados
  pares=filterPairs(pares)
  #print(f"pairs: {pairs}")

  #Recorro cada uno de los pares filtrados y uso los objetos creados
  for par in pares:
    idioma_input.addSentence(par[0])
    idioma_target.addSentence(par[1])


  #Prints necesarios
  print(f"Idioma: {idioma_input.name}, Número de palabras palabras: {idioma_input.n_words}")
  print(f"Idioma: {idioma_target.name}, Número de palabras: {idioma_target.n_words}")

  return idioma_input,idioma_target,pares

# Arquitectura del modelo

### Encoder Architecture

In [19]:
#Encoder Architecture: GRU
class EncoderRNN(nn.Module):
  def __init__(self,input_size,hidden_size,dropout_p=0.1):
    super(EncoderRNN,self).__init__()
    self.hidden_size=hidden_size

    self.embedding=nn.Embedding(num_embeddings=input_size,embedding_dim=hidden_size) #Capa de embedding para transformar a vectores los inputs
    self.gru=nn.GRU(input_size=hidden_size,hidden_size=hidden_size,batch_first=True) #GRU layer que recibe un vector de size == hidden_size (por la embedding layer)
    self.dropout=nn.Dropout(dropout_p) #Dropout para prevenir el overfitting

  def forward(self,input):
    embedded=self.dropout(self.embedding(input)) #Le paso el conjunto de 'tokens' por batch para que la embedding layer le asigne un vector representativo
    output,hidden=self.gru(embedded) #Aquí calculo los hidden states de cada time step (output) y el context vector (hidden state en el último time step)

    return output,hidden

### Decoder Architecture
- El decoder recibirá como input el "context vector" (hidden state, del último time step, que contiene una representación compacta de 'toda' la información), además del token inicial BOS que indica el inicio de la oración.

In [20]:
class DecoderRNN(nn.Module):
  def __init__(self,hidden_size,output_size):
    super(DecoderRNN,self).__init__()

    self.embedding=nn.Embedding(num_embeddings=output_size,embedding_dim=hidden_size) #Embedding layer
    self.gru=nn.GRU(hidden_size,hidden_size,batch_first=True) #GRU Layer
    self.out=nn.Linear(hidden_size,output_size) #Esta es una capa lineal a la que se le aplicará la softmaxt act. function

  def forward(self,encoder_outputs,encoder_hidden,target_tensor=None):
    batch_size=encoder_outputs.size(0) #Obtengo el batch_size
    decoder_input=torch.empty(batch_size,1,dtype=torch.long,device=device).fill_(sos_token) #El input inicial del decoder será el token sos (start of sentence)
    decoder_hidden=encoder_hidden #El hidden state inicial del DECODER será el hidden state del último time step del encoder
    decoder_outputs=[]

    #Forward pass (por cada palabra en una oración): Recibe el de SOS token y el hidden state del último time step del encoder
    for i in range(encoder_outputs.size(1)): #max_length
      decoder_output,decoder_hidden=self.forward_step(decoder_input,decoder_hidden)
      decoder_outputs.append(decoder_output)

      #Teacher forcing
      if target_tensor is not None:
        decoder_input=target_tensor[:,i].unsqueeze(1) #Al usar 'teacher forcing', el target se pasa como input 'siguiente' al decoder

      #Sin teacher forcing: Usa sus 'propias' predicciones para predecir el siguiente input
      else:
        _,topi=decoder_output.topk(1) #Aquí obtengo solo el índice del elemento con mayor valor (y el logit)
        decoder_input=topi.squeeze(-1).detach() #'detach' porque se sobreentiende que es el input siguiente


    decoder_outputs=torch.cat(decoder_outputs,dim=1)
    decoder_outputs=F.log_softmax(decoder_outputs,dim=-1) #Calculo el "log de las probabilidades" a partir de los 'logits' predichos por el modelo

    return decoder_outputs,decoder_hidden,None

  def forward_step(self,input,hidden):
    output=self.embedding(input) #el input pasa por el embedding layer
    output=F.relu(output) #relu activation en el output (embedding)
    output,hidden=self.gru(output,hidden)
    output=self.out(output)

    return output,hidden


# Decoder y Bahdanau Attention

In [102]:
class BahdanauAttention(nn.Module):
  def __init__(self,hidden_size):
    super(BahdanauAttention,self).__init__()
    self.Wa=nn.Linear(hidden_size,hidden_size)
    self.Ua=nn.Linear(hidden_size,hidden_size)
    self.Va=nn.Linear(hidden_size,1)

  def forward(self,query,keys):
    scores=self.Va(torch.tanh(self.Wa(query)+self.Ua(keys)))
    scores=scores.squeeze(2).unsqueeze(1)

    weights=F.softmax(scores,dim=1)
    context=torch.bmm(weights,keys)

    return context,weights

In [103]:
class AttnDecoderRNN(nn.Module):
  def __init__(self,hidden_size,output_size,dropout_p=0.1):
    super(AttnDecoderRNN,self).__init__()
    self.embedding=nn.Embedding(output_size,hidden_size)
    self.attention=BahdanauAttention(hidden_size)
    self.gru=nn.GRU(2*hidden_size,hidden_size,batch_first=True)
    self.out=nn.Linear(hidden_size,output_size)
    self.dropout=nn.Dropout(dropout_p)

  def forward(self,encoder_outputs,encoder_hidden,target_tensor=None):
    batch_size=encoder_outputs.size(0)
    decoder_input=torch.empty(batch_size,1,dtype=torch.long,device=device).fill_(sos_token)
    decoder_hidden=encoder_hidden
    decoder_outputs=[]
    attentions=[]

    for i in range(max_length):
      decoder_output,decoder_hidden,attn_weights=self.forward_step(decoder_input,decoder_hidden,encoder_outputs)

      decoder_outputs.append(decoder_output)
      attentions.append(attn_weights)

      if target_tensor is not None: #Teacher Forcing
        decoder_input=target_tensor[:,i].unsqueeze(1)

      else: #No Teacher Forcing --> Predicciones propias del modelo
        _,topi=decoder_output.topk(1)
        decoder_input=topi.squeeze(-1).detach()

    decoder_outputs=torch.cat(decoder_outputs,dim=1)

    decoder_outputs=F.log_softmax(decoder_outputs,dim=-1)

    attentions=torch.cat(attentions,dim=1)

    return decoder_outputs,decoder_hidden,attentions

  def forward_step(self,input,hidden,encoder_outputs):
    embedded=self.dropout(self.embedding(input)) #convertir a embeddings

    query=hidden.permute(1,0,2)
    context,attn_weights=self.attention(query,encoder_outputs) #calcular los attention weights y el context vector
    input_gru=torch.cat((embedded,context),dim=2) #concatenar el embedded vector y el context vector calculado por los attn weights

    output,hidden=self.gru(input_gru,hidden)
    output=self.out(output)

    return output,hidden,attn_weights

# Creando la data para entrenar al modelo

- Función que convierte las oraciones en una lista de índices

In [23]:
def indexesFromSentence(lang,sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

- Función que añade el token SOS al final y convierte a tensor

In [24]:
def tensorFromSentence(lang,sentence):
    indexes=indexesFromSentence(lang,sentence) #Convierto la oración a índices
    indexes.append(eos_token) #Añado el token EOS
    return torch.tensor(indexes,dtype=torch.long,device=device).view(1,-1)

- Función que mapea un 'pair' (input-target)

In [25]:
def tensorsFromPair(pair):
    input_tensor=tensorFromSentence(input_lang,pair[0]) #El primer elemento del pair es el input lang
    target_tensor=tensorFromSentence(output_lang,pair[1]) #El segundo elemento del pair es el output lang
    return (input_tensor,target_tensor)

- Función para crear el dataloader

In [26]:
def get_dataloader(batch_size):
  input_lang,output_lang,pairs=PrepararData("eng","fra",True)

  n=len(pairs) #Cantidad de oraciones

  input_ids=np.zeros((n,max_length),dtype=np.int32) #Inicialmente, los vectores input/output id serán vectores de 0
  target_ids=np.zeros((n,max_length),dtype=np.int32)

  for idx,(inp,tgt) in enumerate(pairs):
    inp_ids=indexesFromSentence(input_lang,inp) #Listas de tokens índices del input lang
    tgt_ids=indexesFromSentence(output_lang,tgt) #Listas de tokens índices del output lang

    inp_ids.append(eos_token) #Añado el token eos
    tgt_ids.append(eos_token)

    input_ids[idx,:len(inp_ids)]=inp_ids #Al tensor de zeros (input_ids), le coloco los input ids en los primeros "len(input_ids)" índices
    target_ids[idx,:len(tgt_ids)]=tgt_ids #Igual al tensor de zeros para output_id

  #Creo el dataset
  train_data=TensorDataset(torch.LongTensor(input_ids).to(device),
                              torch.LongTensor(target_ids).to(device))

  train_sampler=RandomSampler(train_data)
  train_dataloader=DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

  return input_lang,output_lang,train_dataloader

# Entrenando el modelo

In [104]:
def train_epoch(dataloader_train,encoder,decoder,encoder_optimizer,decoder_optimizer,criterion):
  total_loss = 0

  #Itero sobre el dataloader
  for data in dataloader_train:
    input,target=data

    #El input le paso al encoder --> Genera los outputs en cada time step y el hidden state del último time step
    encoder_outputs,encoder_hidden=encoder(input)

    #El decoder recibe como inputs el encoder_outputs y el último hidden state del encoder y devuelve las log probs (decoder_outputs)
    decoder_outputs,decoder_hidden,_=decoder(encoder_outputs,encoder_hidden,target)

    #Optimizers a 0 para evitar acumulación de gradientes de pasos anteriores
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    #Calculo el loss
    loss=criterion(
          decoder_outputs.view(-1, decoder_outputs.size(-1)),
          target.view(-1)
      )


    #Calculo los gradientes de los pesos a partir del loss
    loss.backward()

    #Actualizo los pesos del modelo en base a esos gradientes
    encoder_optimizer.step()
    decoder_optimizer.step()

    total_loss += loss.item()

  return total_loss / len(dataloader_train)


In [28]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):

    plot_losses = []
    print_loss_total = 0  #Se resetea cada print
    plot_loss_total = 0  #Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss(ignore_index=0) #Ignorar el pad index

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print(f"Época {epoch}, Loss average: {print_loss_avg}")

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0


In [50]:
hidden_size = 128
batch_size = 32

In [51]:
input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

Read 135842 sentence pairs
Idioma: fra, Número de palabras palabras: 5228
Idioma: eng, Número de palabras: 3434
Época 5, Loss average: 2.4382833521064398
Época 10, Loss average: 1.0220397266797332
Época 15, Loss average: 0.5245561697716749
Época 20, Loss average: 0.31354739818324523
Época 25, Loss average: 0.21543348061091847
Época 30, Loss average: 0.16472189253746133
Época 35, Loss average: 0.13508095767154882
Época 40, Loss average: 0.11775858768125622
Época 45, Loss average: 0.10814708167728655
Época 50, Loss average: 0.09984495628431034
Época 55, Loss average: 0.09371503378280784
Época 60, Loss average: 0.08920517302772246
Época 65, Loss average: 0.08514927599865656
Época 70, Loss average: 0.08221261029411338
Época 75, Loss average: 0.08147792295453418
Época 80, Loss average: 0.07901172770866685


# Evaluación del modelo

In [52]:
def evaluar(encoder,decoder,oracion,traduccion_real,input_lang,output_lang,length_max):
  encoder.eval()
  decoder.eval()

  with torch.no_grad():
    oracion_token=tensorFromSentence(input_lang,oracion) #shape [1,10]: 1 muestra, 10 seq length


    #Le paso el vector de tokens al encoder para obtener el hidden state del encoder y pasarle al decoder
    output_encoder,hidden_encoder=encoder(oracion_token) #[1,seq_length,dim_hidden] --> [1,10,128]

    #Decoder
    decoder_outputs,decoder_hidden,decoder_attn=decoder(output_encoder,hidden_encoder)

    #Index de la palabra 'predicha' por el modelo
    _, topi=decoder_outputs.topk(1)
    decoded_ids=topi.squeeze()

    #Predicciones del modelo hasta que se genere el EOS token
    decoded_words=[]
    for idx in decoded_ids:
          if idx.item()==eos_token:
              decoded_words.append('<EOS>')
              break
          decoded_words.append(output_lang.index2word[idx.item()])

  return decoded_words, decoder_attn

# Oraciones de prueba

In [97]:
#Oración de prueba en francés
oraciones=["il est en train de peindre un tableau",
           "il est de loin le meilleur des etudiants",
           "tu es ambitieuse",
           "nous sommes tristes",
           "vous etes chanceuse d avoir un travail",
           "je suis trop vieille pour ce genre de choses",
           "elle fait juste semblant",
           "nous allons toutes a la maison"]

In [98]:
#Traducciones reales de las oraciones
oraciones_target=["he is painting a picture",
           "he is by far the best studeint",
           "you re ambitious",
           "we re sad",
           "you re lucky that you have a job",
           "i m too old for this sort of thing",
          "she s just putting up a front",
          "we re all going home"]

In [99]:
preds=[]
for i in range(len(oraciones)):
  oracion=oraciones[i]
  traduccion_real=oraciones_target[i]

  dec_words,_=evaluar(encoder,decoder,oracion,traduccion_real,input_lang,output_lang,10)
  preds.append(dec_words)

In [100]:
#Imprimo las oraciones predichas
for index,oracion_pred in enumerate(preds):
  oracion_text=""
  for word in oracion_pred:
    oracion_text+=word+" "

  print(f"Oración a traducir: {oraciones[index]}")
  print(f"Oración target: {oraciones_target[index]}")
  print(f"Oración traducida: {oracion_text}")
  print("====")

Oración a traducir: il est en train de peindre un tableau
Oración target: he is painting a picture
Oración traducida: he is painting a picture <EOS> 
====
Oración a traducir: il est de loin le meilleur des etudiants
Oración target: he is by far the best studeint
Oración traducida: he is by far the best student <EOS> 
====
Oración a traducir: tu es ambitieuse
Oración target: you re ambitious
Oración traducida: you re probably tired of cute when you re just 
====
Oración a traducir: nous sommes tristes
Oración target: we re sad
Oración traducida: we re not kind of busy here <EOS> 
====
Oración a traducir: vous etes chanceuse d avoir un travail
Oración target: you re lucky that you have a job
Oración traducida: you re lucky that you have a job <EOS> 
====
Oración a traducir: je suis trop vieille pour ce genre de choses
Oración target: i m too old for this sort of thing
Oración traducida: i m too old for this sort of thing <EOS> 
====
Oración a traducir: elle fait juste semblant
Oración ta

-----